In [92]:
from bs4 import BeautifulSoup
import requests
import re

playerURL = "http://onlinecollegebasketball.org/player/174148"

page = requests.get(playerURL)
soup = BeautifulSoup(page.text, "html.parser")


#infoList = soup.find("table").find_all("tr")[4:11]
fullinfoList = soup.find("table").find_all("tr")
for i in range(len(fullinfoList)):
    if "Age" in fullinfoList[i].text:
        start_index = i
    elif "Perimeter Defense" in fullinfoList[i].text:
        end_index = i + 1

infoList = fullinfoList[start_index:end_index]



def split_number_and_letter(s):
    s = s.replace('↑', '')
    # Split the string into parts that contain numbers, fractions, letters, single quotes, and double quotes
    return re.findall(r'\d+|[¼½]|[^\d¼½\'"]+|[\'"]', s)


'''
def extract_length(array):
    
    length = array[1]
    if "Vertical:" in array:
        if "½" in array or "¼" in array:
            length += array[2]
    else: 
        if "'" in array:
            length += array[2]
            length += array[3]
        if "½" in array or "¼" in array:
            length += array[4]
    return length
'''
def extract_length(array):
    length = ""
    lengthArr = array[1:-4]
    for i in lengthArr:
        length += i
    
    return length



#For Height and Wingspan
def convert_to_inches(height):
    parts = height.split("'")
    if len(parts) == 2:
        feet = int(parts[0])
        inches_str = parts[1].rstrip('"').strip()  # Removing leading/trailing whitespace
        if inches_str:  # Check if inches part is not empty
            if '½' in inches_str:
                inches_str = inches_str.replace('½', '.5')
            elif '¼' in inches_str:
                inches_str = inches_str.replace('¼', '.25')
            inches = float(inches_str)
        else:
            inches = 0
    elif len(parts) == 1:
        feet = int(parts[0])
        inches = 0
    else:
        return None
    return (feet * 12) + inches
    

#For Vertical
def Vert_convert_to_inches(height):
    inches_str = height.rstrip('"')  # Removing the inch symbol
    if '½' in inches_str:
        inches_str = inches_str.replace('½', '.5')
    elif '¼' in inches_str:
        inches_str = inches_str.replace('¼', '.25')
    return float(inches_str)





player_info = {}

for i in range(len(infoList)):
    text = infoList[i].text.strip().replace(" ", "")
    
    if i == 1:
        find_Outside = text.find("O")
        
        separated = split_number_and_letter(text[find_Outside:])
    else:
        separated = split_number_and_letter(text)

    if separated:
        


        if 'Height' in separated[0]:
            separated = separated[-4:]
        if 'Wingspan' in separated[0]:
            separated = separated[-4:]
        if 'Vertical' in separated[0]:
            separated = separated[-4:]
        
        # Handle special cases
        if 'Weight' in separated[0]:
            player_info['Weight'] = float(f"{separated[1]}")
            
            start_idx = 2
            separated[start_idx] = separated[start_idx].replace("lbs.","")

        else:
            start_idx = 0

        # Add remaining key-value pairs to the dictionary
        for j in range(start_idx, len(separated), 2):
            key = separated[j].replace(":", "")
            if j + 1 < len(separated):
                
                value = separated[j + 1]
                player_info[key] = int(value)






player_info["Height_inches"] = convert_to_inches(extract_length(split_number_and_letter(infoList[2].text.strip().replace(" ", ""))))
player_info["Wingspan_inches"] = convert_to_inches(extract_length(split_number_and_letter(infoList[4].text.strip().replace(" ", ""))))

player_info["Vertical_float"] = Vert_convert_to_inches(extract_length(split_number_and_letter(infoList[5].text.strip().replace(" ", ""))))








print(player_info)




{'Age': 21, 'InsideShot': 11, 'BasketballIQ': 15, 'OutsideShot': 10, 'Passing': 11, 'ShootingRange': 9, 'BallHandling': 11, 'Weight': 235.0, 'Finishing': 7, 'Driving': 12, 'Rebounding': 16, 'Strength': 9, 'InteriorDefense': 7, 'Speed': 6, 'PerimeterDefense': 5, 'Stamina': 10, 'Height_inches': 83.5, 'Wingspan_inches': 87.0, 'Vertical_float': 26.0}


In [93]:
def transform_player_data(input_data):
    # Define the mapping from input keys to required keys
    key_mapping = {
        'Age': None,  # Not used in the output
        'InsideShot': 'IS',
        'BasketballIQ': 'IQ',
        'OutsideShot': 'OS',
        'Passing': 'Pass',
        'ShootingRange': 'Rng',
        'BallHandling': 'Hnd',
        'Weight': 'Weight',
        'Finishing': 'Fin',
        'Driving': 'Drv',
        "Rebounding": 'Reb',
        'Strength': 'Str',
        'InteriorDefense': 'IDef',
        'Speed': 'Spd',
        'PerimeterDefense': 'PDef',
        'Stamina': 'Sta',
        "Wingspan_inches": "Wingspan_inches",
        "Height_inches" : "Height_inches",
        "Vertical_float": "Vertical_float"
    }
    
    # Initialize the output dictionary
    output_data = {}
    

    
    # Apply the mapping to transform the input dictionary
    for key, new_key in key_mapping.items():
        if new_key is not None and key in input_data:
            output_data[new_key] = input_data[key]
    
    return output_data


transform_player_data(player_info)

{'IS': 11,
 'IQ': 15,
 'OS': 10,
 'Pass': 11,
 'Rng': 9,
 'Hnd': 11,
 'Weight': 235.0,
 'Fin': 7,
 'Drv': 12,
 'Reb': 16,
 'Str': 9,
 'IDef': 7,
 'Spd': 6,
 'PDef': 5,
 'Sta': 10,
 'Wingspan_inches': 87.0,
 'Height_inches': 83.5,
 'Vertical_float': 26.0}